# OPENROUTESERVICE

API Services Include: **Directions**, **Isochrones**, Time-Distance Matrix, **Geocoding**, **Places Of Interest**, Elevation, Optimization

# NOTES
* for more information, visit python package index and search for openrouteservice
* openrouteservice requires an API key
* openrouteservice coordinates entered longitude, latitude however folium coordinates entered latitude, longitude

# IMPORTS

In [1]:
import openrouteservice as ors
import folium
import PYTHON_API_KEYS

# API KEY REQUIRED
vist https://openrouteservice.org to acquire API key

In [2]:
# enter your openrouteservice api key here as string
ors_key = PYTHON_API_KEYS.API_KEYS.OPENROUTESERVICE

# performs requests to the ORS API services
# client will be used in all examples
client = ors.Client(key=ors_key)

# DIRECTIONS

profile argument (string): specifies the mode of transport to use when calculating directions, options: driving-car, driving-hgv, foot-walking, foot-hiking, cycling-regular, cycling-road, cycling-mountain, cycling-electric

In [3]:
# coordinates from Nashville, TN (-86.781247, 36.163532) to Miami, FL (-80.191850, 25.771645)
# order for coordinates is [lon, lat]
coordinates = [[-86.781247, 36.163532], [-80.191850, 25.771645]]

# directions
route = client.directions(coordinates=coordinates,
                          profile='driving-car',
                          format='geojson')

# map
map_directions = folium.Map(location=[33.77, -84.37], zoom_start=5)

# add geojson to map
folium.GeoJson(route, name='route').add_to(map_directions)

# add layer control to map (allows layer to be turned on or off)
folium.LayerControl().add_to(map_directions)

# display map
map_directions

### view trip information using json data

In [4]:
# distance and duration
print(route['features'][0]['properties']['segments'][0]['distance']*0.000621371, 'miles')
print(route['features'][0]['properties']['segments'][0]['duration']*0.000277778, 'hours\n')

# distances are in meters
# timings are in seconds
print('directions')
for index, i in enumerate(route['features'][0]['properties']['segments'][0]['steps']):
    print(index+1, i, '\n')

912.8104653315 miles
15.2410955262 hours

directions
1 {'distance': 80.8, 'duration': 14.5, 'type': 11, 'instruction': 'Head southeast on St Cloud Alley', 'name': 'St Cloud Alley', 'way_points': [0, 3]} 

2 {'distance': 61.5, 'duration': 14.8, 'type': 0, 'instruction': 'Turn left onto Church Street', 'name': 'Church Street', 'way_points': [3, 5]} 

3 {'distance': 163.5, 'duration': 13.1, 'type': 1, 'instruction': 'Turn right onto 5th Avenue North', 'name': '5th Avenue North', 'way_points': [5, 11]} 

4 {'distance': 126.5, 'duration': 13.0, 'type': 0, 'instruction': 'Turn left onto Commerce Street', 'name': 'Commerce Street', 'way_points': [11, 15]} 

5 {'distance': 1330.7, 'duration': 116.3, 'type': 1, 'instruction': 'Turn right onto 4th Avenue North', 'name': '4th Avenue North', 'way_points': [15, 51]} 

6 {'distance': 1891.4, 'duration': 118.4, 'type': 13, 'instruction': 'Keep right onto 4th Avenue South, US 31A, US 41A', 'name': '4th Avenue South, US 31A, US 41A', 'way_points': [51,

# ISOCHRONES
An isochrone map depicts the area accessible from a point within a certain time threshold.

Reachable or accessible areas can be shown graphically by isochrones. Isochrones are imaginary lines passing through all the places that can be reached from a particular origin within a specified travel time.

In [5]:
# coordinates
coordinates = [[-80.127995, 25.784097]]

# isochrone
isochrone = client.isochrones(locations=coordinates,
                              range_type='time',
                              # 900 seconds, 15 minutes
                              range=[900],
                              attributes=['total_pop'])

# map
map_isochrone = folium.Map(location=[25.784097, -80.127995], tiles='cartodbpositron', zoom_start=12)

# add geojson to map with population
population = isochrone['features'][0]['properties']['total_pop']
folium.GeoJson(isochrone, name='isochrone', tooltip=f'population: {population:,.0f}').add_to(map_isochrone)

# add marker to map
minutes = isochrone['features'][0]['properties']['value']/60
popup_message = f'outline shows areas reachable within {minutes} minutes'
folium.Marker([25.784097, -80.127995], popup=popup_message, tooltip='click').add_to(map_isochrone)

# add layer control to map (allows layer to be turned on or off)
folium.LayerControl().add_to(map_isochrone)

# display map
map_isochrone

# PELIAS GEOCODING

In [6]:
# map
map_geocode = folium.Map(location=[31.7755, -106.4664], tiles='cartodbpositron', zoom_start=13)

# address
address = '4001 E Paisano Dr, El Paso, TX, 79905'

# geocode
geocode = client.pelias_search(text=address, focus_point=list(reversed(map_geocode.location)))

# add marker to map (El Paso Zoo)
for result in geocode['features']:
    folium.Marker(location=list(reversed(result['geometry']['coordinates'])),
                  icon=folium.Icon(icon='building', color='green', prefix='fa'),
                  popup=folium.Popup(result['properties']['name'])).add_to(map_geocode)

# display map
map_geocode

# POIS - PLACES OF INTEREST
filter_category_ids = https://github.com/GIScience/openpoiservice/blob/master/openpoiservice/server/categories/categories.yml

In [7]:
# coordinates
geojson = {"type": "point", "coordinates": [-106.435073, 31.774464]}
coordinates = [31.774464, -106.435073]

# places of interest
pois = client.places(request='pois',
                     geojson=geojson,
                     # buffer searches (in meters) around specified point
                     buffer=2000,
                     # hospital: 206, restaurant: 570
                     filter_category_ids=[206, 570])

# map
map_pois = folium.Map(location=coordinates, tiles='cartodbpositron', zoom_start=14)

# add center point
folium.Marker(coordinates, icon=folium.Icon(color='red')).add_to(map_pois)

# add search area circle
folium.Circle(radius=2000, location=coordinates, color='green').add_to(map_pois)

# add markers to map
for poi in pois['features']:
    folium.Marker(location=list(reversed(poi['geometry']['coordinates'])),
                  icon=folium.Icon(color='blue'),
                  popup=folium.Popup(poi['properties']['osm_tags']['name'])).add_to(map_pois)

# display map
map_pois